In [ ]:
import pandas as pd
import nltk 
import numpy as np
import re
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import twitter_samples
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, hamming_loss


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
def limpiar_encabezado(noticia):
    
    stemmer = PorterStemmer()  # Inicializar PorterStemmer
    stop_words = nltk.corpus.stopwords.words("english")  # Cargar stopwords en inglés
    bad_chars = ["$","USD","€","£","¥","%","&","(",")",",",".","?","¡","¿",'"',"'"]
    noticia = ''.join(i for i in noticia if not i in bad_chars)  # Eliminar caracteres no deseados
    noticia = re.sub(r'https?://[^\s\n\r]+', '', noticia) # Eliminar url
    noticia = re.sub(r'@\w+', '', noticia)  # Eliminar menciones
    noticia = re.sub(r'[^a-zA-Z\s]', '', noticia)  # Eliminar caracteres especiales
    noticia_token = noticia.lower().split()  # Tokenizar el tweet
    noticia_lista = []
    for word in noticia_token:
        if word not in stop_words:
            noticia_lista.append(word)
    return noticia_lista

In [43]:
df= pd.read_csv("financial_news_events.csv", delimiter=",")
df

,Date,Headline,Source,Market_Event,Market_Index,Index_Change_Percent,Trading_Volume,Sentiment,Sector,Impact_Level,Related_Company,News_Url
0,2025-05-21,Nikkei 225 index benefits from a weaker yen,Times of India,Commodity Price Shock,DAX,3.52,166.45,NaN,Technology,High,Goldman Sachs,https://timesofindia.indiatimes.com/business/m...
1,2025-05-18,Government subsidy program gives a lift to the...,Financial Times,Central Bank Meeting,Shanghai Composite,-3.39,57.61,NaN,Retail,Low,ExxonMobil,https://timesofindia.indiatimes.com/business/m...
2,2025-06-25,New housing data release shows a slowdown in m...,The Hindu Business Line,Consumer Confidence Report,Shanghai Composite,-0.05,403.22,Neutral,Retail,Medium,Boeing,https://www.moneycontrol.com/us-markets/sp-500
3,2025-07-21,Massive stock buyback program announced by a c...,The Economist,Commodity Price Shock,NSE Nifty,-2.29,100.11,Positive,Consumer Goods,Low,Samsung Electronics,https://www.cnbc.com/2025/09/automotive-indust...
4,2025-07-23,Government spending bill is expected to stimul...,The Motley Fool,Inflation Data Release,Nasdaq Composite,-3.97,438.22,Negative,Consumer Goods,Low,JP Morgan Chase,https://www.bloomberg.com/australia/asx-200-pe...
...,...,...,...,...,...,...,...,...,...,...,...,...
3019,2025-02-11,Tech Giant's New Product Launch Sparks Sector-...,The Motley Fool,Economic Data Release,DAX,-2.22,212.90,Neutral,Industrials,Low,Samsung Electronics,https://www.marketwatch.com/story/consumer-goo...
3020,2025-08-07,Government spending bill is expected to stimul...,Bloomberg,Geopolitical Event,Euro Stoxx 50,-0.17,105.88,Neutral,Aerospace & Defense,Low,Microsoft,https://www.ft.com/content/consumer-confidence...
3021,2025-05-22,Government spending bill is expected to stimul...,The Economist,Market Rally,Shanghai Composite,4.02,152.28,Negative,Telecommunications,Low,Samsung Electronics,https://www.reuters.com/markets/indices/ftse-100
3022,2025-07-17,Pharmaceutical company's drug approval drives ...,Forbes,Economic Data Release,CAC 40,0.12,53.49,Neutral,Healthcare,Medium,Goldman Sachs,https://www.business-standard.com/internationa...


In [44]:

columns_drop = ['Date', 'News_Url','Market_Index', 'Source', 'Index_Change_Percent','Trading_Volume','Market_Event','Related_Company','Sector','Impact_Level']
df_clean = df.drop(columns=columns_drop)
df_clean.dropna(inplace=True)
df_clean

,Headline,Sentiment
2,New housing data release shows a slowdown in m...,Neutral
3,Massive stock buyback program announced by a c...,Positive
4,Government spending bill is expected to stimul...,Negative
5,Central bank maintains status quo on interest ...,Negative
6,Tech Giant's New Product Launch Sparks Sector-...,Neutral
...,...,...
3019,Tech Giant's New Product Launch Sparks Sector-...,Neutral
3020,Government spending bill is expected to stimul...,Neutral
3021,Government spending bill is expected to stimul...,Negative
3022,Pharmaceutical company's drug approval drives ...,Neutral


In [45]:
df_clean['encabezado_tokenizado'] = df_clean['Headline'].apply(limpiar_encabezado)
df_clean = df_clean.drop(columns=['Headline'])
df_clean['encabezado_tokenizado'] = df_clean['encabezado_tokenizado'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
df_clean = df_clean[df_clean['encabezado_tokenizado'].str.strip() != '']

In [46]:
df_clean

,Sentiment,encabezado_tokenizado
2,Neutral,new housing data release shows slowdown market...
3,Positive,massive stock buyback program announced consum...
4,Negative,government spending bill expected stimulate ec...
5,Negative,central bank maintains status quo interest rates
6,Neutral,tech giants new product launch sparks sectorwi...
...,...,...
3019,Neutral,tech giants new product launch sparks sectorwi...
3020,Neutral,government spending bill expected stimulate ec...
3021,Negative,government spending bill expected stimulate ec...
3022,Neutral,pharmaceutical companys drug approval drives s...


In [47]:

# Preparar datos
X = df_clean['encabezado_tokenizado']  # tus tweets tokenizados
y = df_clean['Sentiment']  # las 3 etiquetas

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [48]:
# Vectorización
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
modelo = NeuralNetClassifier(input_size=X_train_vec.shape[1], hidden_size=100, output_size=len(y.unique()), learning_rate=0.001, epochs=10)

In [50]:
modelo.fit(X_train_vec, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [51]:
# Predecir
y_pred = modelo.predict(X_test_vec)

# Evaluar
print("Hamming Loss:", hamming_loss(y_test, y_pred))

print(classification_report(y_test, y_pred))

Hamming Loss: 0.6869244935543278
              precision    recall  f1-score   support

    Negative       0.33      0.35      0.34       187
     Neutral       0.29      0.34      0.31       174
    Positive       0.32      0.25      0.28       182

    accuracy                           0.31       543
   macro avg       0.31      0.31      0.31       543
weighted avg       0.31      0.31      0.31       543

